# GOAL: Predict the shopping time for a given trip

In [ ]:
! conda install -n instacart seaborn

Using Anaconda Cloud api site https://api.anaconda.org
Fetching package metadata: ....
Solving package specifications: .........

Package plan for installation in environment /Users/louis/anaconda/envs/instacart:

The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    seaborn-0.7.0              |           py27_0         264 KB

The following NEW packages will be INSTALLED:

    seaborn: 0.7.0-py27_0

Proceed ([y]/n)? 

In [ ]:
y

In [1]:
# import dependencies
import pandas as pd
import csv
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import calendar #necessary to convert a timestamp into a date
#from datetime import datetime
import time
import seaborn as sns
from __future__ import division

ImportError: No module named seaborn

In [ ]:
# Run processing methods
%run '/Users/louis/Documents/DataScience/Instacart/processing_instacart.py'

In [3]:
# Load train and test trips
path_train = path_train = "/Users/louis/Documents/DataScience/Instacart/data/train_trips.csv"
data_trips_train = pd.read_csv(path_train)
data_raw_train = pd.read_csv(path_train)
data_raw_train = pd.read_csv(path_train)
# Load test set
path_test = "/Users/louis/Documents/DataScience/Instacart/data/test_trips.csv"
data_trips_test = pd.read_csv(path_test)
# Load order items
path_items = "/Users/louis/Documents/DataScience/Instacart/data/order_items.csv"
data_items = pd.read_csv(path_items)

In [4]:
# Split training set into X and y - get exact same format for X_train and X_test
#hash_output = preprocess_train(data_trips_train)
#X_train = hash_output['X_train']
#y_train = hash_output['y_train']
data = process_data(data_trips_train, data_items, data_raw_train)
data_trips_test['shopping_started_at'] = data_trips_test['shopping_started_at'].map(lambda date: datetime.strptime(date, '%Y-%m-%d %H:%M:%S'))
#X_train = process_data(X_train,data_items,data_raw_train)
data.head()

,fulfillment_model,store_id,trip_length,hour,day_of_week,department_count,quantity,shopper_efficiency
trip_id,,,,,,,,
3119519,1,6,1620,7,1,11,39.0,<3000
3119513,1,1,2160,7,1,8,183.0,<2500
3119516,1,1,1080,7,1,5,9.0,<2000
3119792,1,1,5160,7,1,10,92.0,<2500
3119922,1,1,5340,7,1,7,186.0,<2000


In [5]:
# dummify and create y
y = data['trip_length']
del data['trip_length']

In [6]:
data_dumm = pd.get_dummies(data, columns = ['hour', 'day_of_week','store_id', 'shopper_efficiency'])

# Basic model

In [7]:
from sklearn import linear_model
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn import metrics

In [8]:
# Create test and train set
X_train, X_test, y_train, y_test = cross_validation.train_test_split(data_dumm, y, test_size=0.2, random_state=0)

In [9]:
y_test.shape

(23413,)

In [10]:
lr = linear_model.LinearRegression(fit_intercept = True)
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [11]:
# predict
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [16]:
def compute_metric(y_train_pred, y_train):
    delta = y_train_pred - y_train
    delta = delta.map(lambda x: abs(x))
    moy = y_train.mean()
    delta = delta.map(lambda x: x/moy)
    return delta.mean()

compute_metric(y_train_pred, y_train)

0.33510546506168104

In [18]:
print 'Average error on train: %s' %compute_metric(y_train_pred, y_train)
print 'Average error on test: %s' %compute_metric(y_test_pred, y_test)

Average error on train: 0.335105465062
Average error on test: 0.332858426527


# Model including distances of departments

In [66]:
def dummify_pivot(x):
    if np.isnan(x):
        return 0
    else:
        return 1
# define departments to keep
departments_to_keep = data_items['department_name'].value_counts(dropna = False).index.values[0:200]
# Generate pivot and create dummy columns for each department
pivot_department = pd.pivot_table(data_items, index = 'trip_id', columns = 'department_name', aggfunc = "count")['item_id']
pivot_department = pivot_department[departments_to_keep]
dummies_departments = pivot_department.applymap(dummify_pivot)
polynom_departments = preprocessing.PolynomialFeatures(degree = 2, interaction_only = True)
dummies_departments_interaction = polynom_departments.fit_transform(dummies_departments)
dummies_departments_interaction.shape


(128710, 3742)

In [67]:
dumm_departments_df = pd.DataFrame(dummies_departments_interaction)
dumm_departments_df.index = dummies_departments.index
dumm_departments_df.head()

,0,1,2,3,4,5,6,7,8,9,...,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741
trip_id,,,,,,,,,,,,,,,,,,,,,
3119513,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119516,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119518,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119519,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119520,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
data_dumm_dpt = data_dumm.merge(dumm_departments_df, left_index = True, right_index = True, how = 'left')
data_dumm_dpt.head()

,fulfillment_model,department_count,quantity,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,...,3732,3733,3734,3735,3736,3737,3738,3739,3740,3741
trip_id,,,,,,,,,,,,,,,,,,,,,
3119519,1,11,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119513,1,8,183.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119516,1,5,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119792,1,10,92.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3119922,1,7,186.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
# Create test and train set
X_train, X_test, y_train, y_test = cross_validation.train_test_split(data_dumm_dpt, y, test_size=0.2, random_state=0)

In [70]:
lr = linear_model.LinearRegression(fit_intercept = True)
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [64]:
# predict
y_train_pred = lr.predict(X_train)
y_test_pred = lr.predict(X_test)

In [65]:
print 'Average error on train: %s' %compute_metric(y_train_pred, y_train)
print 'Average error on test: %s' %compute_metric(y_test_pred, y_test)

Average error on train: 0.333124413587
Average error on test: 0.331300922471


In [53]:
coeff_df = pd.DataFrame(lr.coef_)

In [57]:
coeff_df['feature'] = X_train.columns.values
coeff_df.columns = ['coeff', 'feature']

In [58]:
coeff_df.sort_values('coeff', ascending = False)

,coeff,feature
26,8.066498e+10,day_of_week_6
21,8.066498e+10,day_of_week_1
20,8.066498e+10,day_of_week_0
24,8.066498e+10,day_of_week_4
25,8.066498e+10,day_of_week_5
23,8.066498e+10,day_of_week_3
22,8.066498e+10,day_of_week_2
30,3.662686e+10,store_id_6
39,3.662686e+10,store_id_126
38,3.662686e+10,store_id_123


In [ ]:
TODO:
    1. Check correlation of of y to each department - include most relevant departments
    2. Check correlation of y to each department interaction - include most relevant departments